In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/jcmor/desktop/Ironhack/ironhack_service_account_big_query.json"
from google.cloud import bigquery

# Instantiates a client for BigQuery Service
client = bigquery.Client()

In [2]:
query1 = '''

SELECT 
        author.au_id AS AUTHOR_ID,
        author.au_lname AS AUTHOR_LAST_NAME, 
        author.au_fname AS AUTHOR_FIRST_NAME, 
        titles.title AS TITLE, 
        publishers.pub_name AS PUBLISHER,
        
FROM 
    `ironhack-data-analytics-265219.publications.authors` author
JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
ON 
    author.au_id = titleauthor.au_id
JOIN 
    `ironhack-data-analytics-265219.publications.titles` titles
ON 
    titles.title_id = titleauthor.title_id
JOIN 
    `ironhack-data-analytics-265219.publications.publishers` publishers
ON 
    publishers.pub_id = titles.pub_id
'''
query_job = client.query(query=query1)
df=query_job.to_dataframe()
df.head()

,AUTHOR_ID,AUTHOR_LAST_NAME,AUTHOR_FIRST_NAME,TITLE,PUBLISHER
0,899-46-2035,Ringer,Anne,Is Anger the Enemy?,New Moon Books
1,846-92-7186,Hunter,Sheryl,Secrets of Silicon Valley,Algodata Infosystems
2,213-46-8915,Green,Marjorie,The Busy Executive's Database Guide,Algodata Infosystems
3,267-41-2394,O'Leary,Michael,Cooking with Computers: Surreptitious Balance ...,Algodata Infosystems
4,267-41-2394,O'Leary,Michael,"Sushi, Anyone?",Binnet & Hardley


In [3]:
query2 = '''
SELECT          
        author.au_id AS AUTHOR_ID,
        author.au_lname AS AUTHOR_LAST_NAME, 
        author.au_fname AS AUTHOR_FIRST_NAME, 
        publishers.pub_name AS PUBLISHER,
        COUNT(titles.title_id) AS TITLE_COUNT 
        
FROM 
    `ironhack-data-analytics-265219.publications.authors` author
JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
ON 
    author.au_id = titleauthor.au_id
JOIN 
    `ironhack-data-analytics-265219.publications.titles` titles
ON 
    titles.title_id = titleauthor.title_id
JOIN 
    `ironhack-data-analytics-265219.publications.publishers` publishers
ON 
    publishers.pub_id = titles.pub_id
GROUP BY 
    1,2,3,4
ORDER BY
    TITLE_COUNT DESC
'''
query_job = client.query(query=query2)
df=query_job.to_dataframe()
df

,AUTHOR_ID,AUTHOR_LAST_NAME,AUTHOR_FIRST_NAME,PUBLISHER,TITLE_COUNT
0,998-72-3567,Ringer,Albert,New Moon Books,2
1,807-91-6654,Panteley,Sylvia,Binnet & Hardley,1
2,722-51-5454,DeFrance,Michel,Binnet & Hardley,1
3,712-45-1867,del Castillo,Innes,Binnet & Hardley,1
4,899-46-2035,Ringer,Anne,New Moon Books,1
5,899-46-2035,Ringer,Anne,Binnet & Hardley,1
6,172-32-1176,White,Johnson,New Moon Books,1
7,486-29-1786,Locksley,Charlene,New Moon Books,1
8,486-29-1786,Locksley,Charlene,Algodata Infosystems,1
9,846-92-7186,Hunter,Sheryl,Algodata Infosystems,1


In [4]:
query3 = '''
SELECT          
        author.au_id AS AUTHOR_ID,
        author.au_lname AS AUTHOR_LAST_NAME, 
        author.au_fname AS AUTHOR_FIRST_NAME,
        SUM(titles.ytd_sales) AS TOTAL,
FROM 
    `ironhack-data-analytics-265219.publications.authors` author
JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
ON 
    author.au_id = titleauthor.au_id
JOIN 
    `ironhack-data-analytics-265219.publications.titles` titles
ON 
    titles.title_id = titleauthor.title_id
GROUP BY 
    1,2,3
ORDER BY 
    TOTAL
    DESC
LIMIT
    3
'''
query_job = client.query(query=query3)
df=query_job.to_dataframe()
df

,AUTHOR_ID,AUTHOR_LAST_NAME,AUTHOR_FIRST_NAME,TOTAL
0,899-46-2035,Ringer,Anne,24291
1,213-46-8915,Green,Marjorie,22817
2,722-51-5454,DeFrance,Michel,22246


In [5]:
query4 = '''
SELECT          
        author.au_id AS AUTHOR_ID,
        author.au_lname AS AUTHOR_LAST_NAME, 
        author.au_fname AS AUTHOR_FIRST_NAME,
        COALESCE(SUM(titles.ytd_sales), 0) AS TOTAL,
FROM 
    `ironhack-data-analytics-265219.publications.authors` author
LEFT JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
ON 
    author.au_id = titleauthor.au_id
LEFT JOIN 
    `ironhack-data-analytics-265219.publications.titles` titles
ON 
    titles.title_id = titleauthor.title_id
GROUP BY 
    1,2,3
ORDER BY 
    TOTAL
    DESC
'''
query_job = client.query(query=query4)
df=query_job.to_dataframe()
df

,AUTHOR_ID,AUTHOR_LAST_NAME,AUTHOR_FIRST_NAME,TOTAL
0,899-46-2035,Ringer,Anne,24291
1,213-46-8915,Green,Marjorie,22817
2,722-51-5454,DeFrance,Michel,22246
3,648-92-1872,Blotchet-Halls,Reginald,15096
4,238-95-7766,Carson,Cheryl,8780
5,267-41-2394,O'Leary,Michael,7971
6,724-80-9391,MacFeather,Stearns,4251
7,846-92-7186,Hunter,Sheryl,4095
8,427-17-2319,Dull,Ann,4095
9,672-71-3249,Yokomoto,Akiko,4095


In [6]:
query5 = '''
SELECT   
    author.au_id AS AUTHOR_ID,
    author.au_lname AS AUTHOR_LAST_NAME,
    author.au_fname AS AUTHOR_FIRST_NAME,
    SUM((titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100)+ (titles.advance * titleauthor.royaltyper / 100)) PROFIT,
FROM 
    `ironhack-data-analytics-265219.publications.authors` author
JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
ON 
    author.au_id = titleauthor.au_id
JOIN 
    `ironhack-data-analytics-265219.publications.titles` titles
ON 
    titles.title_id = titleauthor.title_id
JOIN 
    `ironhack-data-analytics-265219.publications.sales` sales
ON
    sales.title_id = titles.title_id
GROUP BY 
    1,2,3
ORDER BY
    PROFIT
    DESC
LIMIT
    3
'''
query_job = client.query(query=query5)
df=query_job.to_dataframe()
df

,AUTHOR_ID,AUTHOR_LAST_NAME,AUTHOR_FIRST_NAME,PROFIT
0,722-51-5454,DeFrance,Michel,22521.528
1,213-46-8915,Green,Marjorie,14162.110
2,899-46-2035,Ringer,Anne,12128.132
